## **Accounting Fraud Detection using Normalizing-Flows**

In [1]:
!pip install nflows --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.8/45.8 kB 2.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 92.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 61.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 37.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 30.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.3 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 2.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 10.5 MB/s eta 0:

In [34]:
import numpy as np
import torch
import torch.optim as optim
from torch.utils.data import Dataset, TensorDataset, DataLoader, random_split
from sklearn.preprocessing import StandardScaler, RobustScaler
import os

# nflows is a specialized library for normalizing flows.
from nflows.flows.base import Flow
from nflows.distributions.normal import StandardNormal
from nflows.transforms.base import CompositeTransform
from nflows.transforms.autoregressive import MaskedAffineAutoregressiveTransform
from nflows.transforms.permutations import RandomPermutation
from nflows.transforms.normalization import BatchNorm, ActNorm

# --- Data Simulation and Preprocessing (Unchanged) ---
def generate_synthetic_data(n_samples_normal=1000, n_samples_fraud=50):
    """Generates synthetic normal and fraudulent financial data."""
    mean_normal = np.array([0.5, 0.2, 0.7, 0.4, 0.6])
    cov_normal = np.array([
        [0.02, 0.01, 0.005, 0.005, 0.005],
        [0.01, 0.03, 0.01, 0.005, 0.005],
        [0.005, 0.01, 0.02, 0.005, 0.005],
        [0.005, 0.005, 0.005, 0.03, 0.01],
        [0.005, 0.005, 0.005, 0.01, 0.02]
    ])
    normal_data = np.random.multivariate_normal(mean_normal, cov_normal, n_samples_normal)

    mean_fraud = np.array([0.1, 0.8, 0.2, 0.9, 0.1])
    cov_fraud = np.array([
        [0.05, -0.02, 0.01, -0.03, 0.01],
        [-0.02, 0.06, -0.01, 0.04, -0.01],
        [0.01, -0.01, 0.04, -0.02, 0.01],
        [-0.03, 0.04, -0.02, 0.07, -0.02],
        [0.01, -0.01, 0.01, -0.02, 0.05]
    ])
    fraud_data = np.random.multivariate_normal(mean_fraud, cov_fraud, n_samples_fraud)
    return normal_data, fraud_data

"""
normal_data, fraud_data = generate_synthetic_data()
X_train = normal_data.astype(np.float32)
X_test = np.vstack([normal_data[:200], fraud_data]).astype(np.float32)
y_test = np.hstack([np.zeros(200), np.ones(fraud_data.shape[0])])

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)
"""

'\nnormal_data, fraud_data = generate_synthetic_data()\nX_train = normal_data.astype(np.float32)\nX_test = np.vstack([normal_data[:200], fraud_data]).astype(np.float32)\ny_test = np.hstack([np.zeros(200), np.ones(fraud_data.shape[0])])\n\nscaler = StandardScaler()\nX_train_scaled = scaler.fit_transform(X_train)\nX_test_scaled = scaler.transform(X_test)\n'

In [28]:
import pandas as pd

df = pd.read_csv(r"/kaggle/input/accounting-fraud/uscecchini28.csv")
df.tail()
print(df.columns)

# Check for infinite values in the entire dataframe
inf_counts = df.isin([np.inf, -np.inf]).sum()
print("Columns with infinite values:")
print(inf_counts[inf_counts > 0])

# df.replace([np.inf, -np.inf], np.nan, inplace=True)

Index(['fyear', 'gvkey', 'sich', 'insbnk', 'understatement', 'option',
       'p_aaer', 'new_p_aaer', 'misstate', 'act', 'ap', 'at', 'ceq', 'che',
       'cogs', 'csho', 'dlc', 'dltis', 'dltt', 'dp', 'ib', 'invt', 'ivao',
       'ivst', 'lct', 'lt', 'ni', 'ppegt', 'pstk', 're', 'rect', 'sale',
       'sstk', 'txp', 'txt', 'xint', 'prcc_f', 'dch_wc', 'ch_rsst', 'dch_rec',
       'dch_inv', 'soft_assets', 'ch_cs', 'ch_cm', 'ch_roa', 'issue', 'bm',
       'dpi', 'reoa', 'EBIT', 'ch_fcf'],
      dtype='object')
Columns with infinite values:
Series([], dtype: int64)


In [9]:
from scipy.stats import chisquare
from sklearn.model_selection import train_test_split

# --- raw variables and Benford's theoretical probabilities ---
raw_variables = ['at', 'ap', 'ceq', 'che', 'csho', 'dltt', 'dp', 'ni', 'ppegt',
                  'pstk', 're', 'rect', 'sale', 'lt', 'xint', 'ivao', 'dltis', 'sstk']

# Theoretical probabilities for digits 1-9
benford_probs = np.array([np.log10(1 + 1 / d) for d in range(1, 10)])

# --- function to calculate the Benford's Law Chi-squared statistic for a row ---
def calculate_benford_chi2(row):
    """Calculates the Chi-squared statistic for a given row based on Benford's Law."""

    # Extract first digits from the row's variables, excluding NaNs and zeros
    first_digits = [
        int(str(abs(x))[0]) for x in row[raw_variables]
        if pd.notna(x) and x != 0
    ]

    if not first_digits:
        return np.nan

    # Count the occurrences of each digit
    digit_counts = pd.Series(first_digits).value_counts().sort_index()

    # Create an observed counts array for all 9 digits
    observed = np.zeros(9)
    for i, count in digit_counts.items():
        if 1 <= i <= 9:
            observed[i-1] = count

    # Expected counts based on Benford's Law
    n = len(first_digits)
    expected = benford_probs * n

    # Calculate Chi-squared statistic, handling zero values
    expected_safe = expected + 1e-10
    chi2_stat = np.sum((observed - expected)**2 / expected_safe)

    return chi2_stat

# --- Add the new feature column to the DataFrame ---
# df['benford_chi2'] = df.apply(calculate_benford_chi2, axis=1)



In [29]:
from sklearn.model_selection import train_test_split

# Separate all normal and fraud data
df_normal = df[df['misstate'] == 0]
df_fraud = df[df['misstate'] == 1]

# Define features
features = ['act', 'ap', 'at', 'ceq', 'che',
            'cogs', 'csho', 'dlc', 'dltis', 'dltt', 'dp', 'ib', 'invt', 'ivao',
            'ivst', 'lct', 'lt', 'ni', 'ppegt', 'pstk', 're', 'rect', 'sale',
            'sstk', 'txp', 'txt', 'xint', 'prcc_f', 'dch_wc', 'ch_rsst', 'dch_rec',
            'dch_inv', 'soft_assets', 'ch_cs', 'ch_cm', 'ch_roa', 'issue', 'bm',
            'dpi', 'reoa', 'ch_fcf']

# Split the NORMAL data first into a training and a testing set
X_normal = df_normal[features].dropna()
y_normal = df_normal.loc[X_normal.index]['misstate']

X_train, X_test_normal, y_train, y_test_normal = train_test_split(
    X_normal, y_normal, test_size=0.05, random_state=42
)

# Now, fit the scaler ONLY on the training data
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train) # Fit ONLY on X_train

zero_variance_cols = X_train.columns[scaler.scale_ == 0]
if len(zero_variance_cols) > 0:
    print("Found columns with zero variance (standard deviation):")
    print(list(zero_variance_cols))
else:
    print("No columns with zero variance found.")

# Prepare the fraud data
X_test_fraud = df_fraud[features].dropna()
y_test_fraud = df_fraud.loc[X_test_fraud.index]['misstate']

# Transform the test sets using the FITTED scaler
X_test_normal_scaled = scaler.transform(X_test_normal)
X_test_fraud_scaled = scaler.transform(X_test_fraud)

# Combine to create the final test set
X_test_scaled = np.vstack([X_test_fraud_scaled, X_test_normal_scaled])
y_test = pd.concat([y_test_fraud, y_test_normal])

No columns with zero variance found.


In [30]:
print(f"Original training data shape: {X_train_scaled.shape}")

# Define a threshold for extreme values
threshold = 200

# Find the indices of all rows that contain a value exceeding the threshold
outlier_indices = np.where(np.abs(X_train_scaled) > threshold)[0]
# Get a unique set of indices, as a row might have multiple outliers
outlier_indices = np.unique(outlier_indices)

print(f"Found {len(outlier_indices)} rows with extreme outlier values.")

# Remove the outlier rows
X_train_scaled = np.delete(X_train_scaled, outlier_indices, axis=0)

print(f"New training data shape after filtering: {X_train_scaled.shape}")

Original training data shape: (119295, 41)
Found 0 rows with extreme outlier values.
New training data shape after filtering: (119295, 41)


In [36]:
# Convert to PyTorch Tensors
X_train_tensor = torch.from_numpy(X_train_scaled.astype(np.float64))
y_train_tensor = torch.from_numpy(y_train.values.astype(np.float64))
X_test_tensor = torch.from_numpy(X_test_scaled.astype(np.float64))

# Check for NaNs and infinite values in tensors
if torch.isnan(X_train_tensor).any() or torch.isinf(X_train_tensor).any():
    print("NaN or infinite values found in X_train_tensor!")
if torch.isnan(y_train_tensor).any() or torch.isinf(y_train_tensor).any():
    print("NaN or infinite values found in y_train_tensor!")
if torch.isnan(X_test_tensor).any() or torch.isinf(X_test_tensor).any():
    print("NaN or infinite values found in X_test_tensor!")

# Create datasets from the scaled tensors
train_dataset = TensorDataset(X_train_tensor)

# Create a validation split
train_size = int(0.8 * len(train_dataset))
val_size = len(train_dataset) - train_size
train_dataset, val_dataset = random_split(train_dataset, [train_size, val_size])

def weights_init(m):
    """
    Applies a custom weight initialization to the model's layers.
    Initializes linear layers with a normal distribution (mean=0, std=0.02)
    and sets biases to zero.
    """
    if isinstance(m, torch.nn.Linear):
        torch.nn.init.normal_(m.weight.data, 0.0, 0.02)
        torch.nn.init.constant_(m.bias.data, 0.0)

# --- Define the Flow-based Model using nflows ---
def create_nflows_model(input_dim, num_layers=6, hidden_dim=256):
    """Creates a RealNVP-style model using the nflows library."""
    base_dist = StandardNormal(shape=[input_dim])

    transforms = []
    for _ in range(num_layers):
        transforms.append(RandomPermutation(features=input_dim))
        transforms.append(MaskedAffineAutoregressiveTransform(
            features=input_dim,
            hidden_features=hidden_dim
        ))
        transforms.append(ActNorm(features=input_dim))

    transform = CompositeTransform(transforms)

    flow = Flow(transform, base_dist)
    return flow

In [37]:
# --- Forensic Debugging Cell ---
print("--- Starting Forensic Debugging ---")
train_dataset = TensorDataset(X_train_tensor)

# Use the exact same (simplified) settings that are failing
input_dim = X_train_scaled.shape[1]
num_coupling_layers = 8
hidden_dim = 256
batch_size = 256 # Ensure this matches your training batch_size

# Re-create the DataLoader to ensure we get a consistent first batch
# NOTE: Set shuffle=False to make the test repeatable.
train_loader_debug = DataLoader(train_dataset, batch_size=batch_size, shuffle=False)

# Get the VERY FIRST batch of data
try:
    first_batch = next(iter(train_loader_debug))[0]
    print(f"\nSuccessfully loaded one batch of size: {first_batch.shape}")
except StopIteration:
    print("Error: DataLoader is empty. Check your train_dataset.")
    # Stop here if there's no data

# Inspect the batch itself for any issues post-scaling and loading
print(f"Batch Stats: Min={first_batch.min():.4f}, Max={first_batch.max():.4f}, Mean={first_batch.mean():.4f}")
if torch.isnan(first_batch).any() or torch.isinf(first_batch).any():
    print("!!! FATAL ERROR: The batch itself contains NaNs or Infs before entering the model!")
else:
    print("✅ Batch data appears clean (no NaNs or Infs).")

# Create a fresh model instance and run a single forward pass
model_debug = create_nflows_model(input_dim, num_coupling_layers, hidden_dim)
model_debug.double()
model_debug.eval() # Set to evaluation mode for a clean forward pass

print("\n--- Running Forward Pass on Single Batch ---")
with torch.no_grad():
    try:
        log_probs = model_debug.log_prob(inputs=first_batch)

        # 6. Check for issues in the model's output
        nan_mask = torch.isnan(log_probs)
        inf_mask = torch.isinf(log_probs)

        if nan_mask.any():
            print(f"🔴 FAILURE: Found {nan_mask.sum()} NaN(s) in the log_prob output!")
            # Find the specific row(s) in the batch that caused the NaN
            problem_indices = nan_mask.nonzero(as_tuple=True)[0]
            print(f"Data row at index {problem_indices[0]} in the batch caused the NaN.")
            print("\n--- Problematic Data Point ---")
            print(first_batch[problem_indices[0]])
            print("------------------------------")

        elif inf_mask.any():
            print(f"🔴 FAILURE: Found {inf_mask.sum()} Inf(s) in the log_prob output!")
            problem_indices = inf_mask.nonzero(as_tuple=True)[0]
            print(f"Data row at index {problem_indices[0]} in the batch caused the Inf.")
            print("\n--- Problematic Data Point ---")
            print(first_batch[problem_indices[0]])
            print("------------------------------")
        else:
            print(f"✅ SUCCESS: The forward pass is clean.")
            print(f"Log Probs Stats: Min={log_probs.min():.4f}, Max={log_probs.max():.4f}")
            print("This suggests the NaN may be occurring in the backward pass (gradient calculation).")


    except Exception as e:
        print(f"\nAn exception occurred during the forward pass: {e}")

# Clean up to avoid interfering with the main training loop
del model_debug, train_loader_debug

--- Starting Forensic Debugging ---

Successfully loaded one batch of size: torch.Size([256, 41])
Batch Stats: Min=-6.2611, Max=35.7735, Mean=-0.0104
✅ Batch data appears clean (no NaNs or Infs).

--- Running Forward Pass on Single Batch ---
✅ SUCCESS: The forward pass is clean.
Log Probs Stats: Min=-190.3461, Max=-155.3461
This suggests the NaN may be occurring in the backward pass (gradient calculation).


In [39]:
import torch.nn as nn

# --- Training the Model ---
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

input_dim = X_train_scaled.shape[1]
num_coupling_layers = 15
hidden_dim = 512
learning_rate = 1e-5
batch_size = 256
epochs = 100
patience = 10 # For early stopping
best_model_path = "best_model.pth"
weight_decay = 1e-5

if torch.isnan(torch.tensor(X_train_scaled)).any():
    raise ValueError("Input data contains NaN values. Please clean the data before training.")
else:
    print("Data integrity check passed: No NaN values found in input data.")

model = create_nflows_model(input_dim, num_coupling_layers, hidden_dim)
model.double()
model.apply(weights_init)

if torch.cuda.device_count() > 1:
    print(f"Let's use {torch.cuda.device_count()} GPUs!")
    model = nn.DataParallel(model)
model.to(device)
print(f"Using device: {device}")

optimizer = optim.Adam(model.parameters(), lr=learning_rate, weight_decay=weight_decay)

# Create DataLoaders for train and validation sets
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size)

print("Starting model training...")

best_val_loss = float('inf')
early_stopping_counter = 0

for epoch in range(epochs):
    # Training phase
    model.train()
    train_loss = 0.0
    for (batch,) in train_loader:  
        batch = batch.to(device)
        optimizer.zero_grad()
        if isinstance(model, nn.DataParallel):
            loss = -model.module.log_prob(inputs=batch).mean()
        else:
            loss = -model.log_prob(inputs=batch).mean()
        
        if torch.isnan(loss):
            print("NaN loss detected. Stopping training.")
            break
            
        loss.backward()

        # gradient clipping for stabilization
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=0.5)

        optimizer.step()
        train_loss += loss.item()
    
    if torch.isnan(loss): # Break outer loop if NaN was detected
        break
        
    avg_train_loss = train_loss / len(train_loader)

    # Validation phase
    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for (batch,) in val_loader:
            batch = batch.to(device)
            if isinstance(model, nn.DataParallel):
                loss = -model.module.log_prob(inputs=batch).mean()
            else:
                loss = -model.log_prob(inputs=batch).mean()
            val_loss += loss.item()
    avg_val_loss = val_loss / len(val_loader)

    if (epoch + 1) % 10 == 0:
        print(f"Epoch {epoch + 1}, Train Loss: {avg_train_loss:.4f}, Val Loss: {avg_val_loss:.4f}")

    if avg_val_loss is None or torch.isnan(torch.tensor(avg_val_loss)):
        print(f"Validation loss is None at epoch {epoch + 1}. Stopping training.")
        break

    # Early stopping and best model saving
    if avg_val_loss < best_val_loss:
        best_val_loss = avg_val_loss
        state_to_save = model.module.state_dict() if isinstance(model, nn.DataParallel) else model.state_dict()
        torch.save(state_to_save, best_model_path)
        early_stopping_counter = 0
        print(f"  -> New best model saved with val_loss: {best_val_loss:.4f}")
    else:
        early_stopping_counter += 1

    if early_stopping_counter >= patience:
        print(f"\nEarly stopping triggered after {epoch + 1} epochs.")
        break

print("\nTraining finished.")


Data integrity check passed: No NaN values found in input data.
Let's use 2 GPUs!
Using device: cuda
Starting model training...
  -> New best model saved with val_loss: 26.8816
  -> New best model saved with val_loss: -16.1035
  -> New best model saved with val_loss: -24.1473
  -> New best model saved with val_loss: -28.8335
  -> New best model saved with val_loss: -32.3947
  -> New best model saved with val_loss: -35.2613
  -> New best model saved with val_loss: -37.7187
  -> New best model saved with val_loss: -39.5027
  -> New best model saved with val_loss: -41.1665
Epoch 10, Train Loss: -41.9849, Val Loss: -42.8530
  -> New best model saved with val_loss: -42.8530
  -> New best model saved with val_loss: -44.2922
  -> New best model saved with val_loss: -45.7391
  -> New best model saved with val_loss: -46.6929
  -> New best model saved with val_loss: -47.5988
  -> New best model saved with val_loss: -49.0684
  -> New best model saved with val_loss: -49.8541
  -> New best model sa

In [45]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.set_default_dtype(torch.float64)

model = create_nflows_model(input_dim, num_coupling_layers, hidden_dim)
model.double()

if os.path.exists(best_model_path):
    print(f"\nLoading best model from '{best_model_path}' for evaluation.")
    state_dict = torch.load(best_model_path, map_location=device)

    # Handle the 'module.' prefix if the model was saved using DataParallel
    if next(iter(state_dict)).startswith('module.'):
        new_state_dict = OrderedDict()
        for k, v in state_dict.items():
            name = k[7:] # remove `module.`
            new_state_dict[name] = v
        model.load_state_dict(new_state_dict)
    else:
        model.load_state_dict(state_dict)
else:
    print("\nWarning: No saved model found. Cannot perform evaluation.")
    exit()

model.to(device)

# Wrap with DataParallel if using multiple GPUs
if device.type == 'cuda' and torch.cuda.device_count() > 1:
    print("Wrapping loaded model with DataParallel for evaluation.")
    model = torch.nn.DataParallel(model)

model.eval()


def detect_anomalies(model, data_loader, threshold_percentile=5):
    """Calculates log probabilities and flags anomalies."""
    model.eval()
    log_probs_list = []
    
    # Determine if the model is wrapped with DataParallel
    is_parallel = isinstance(model, nn.DataParallel)
    
    with torch.no_grad():
        for (batch,) in data_loader:
            # Move data to the correct device
            batch = batch.to(device)
            
            if is_parallel:
                log_probs = model.module.log_prob(inputs=batch)
            else:
                log_probs = model.log_prob(inputs=batch)
            
            # Move results back to CPU for concatenation
            log_probs_list.append(log_probs.cpu())
            
    log_probs_tensor = torch.cat(log_probs_list)

    # We need the full original training data to set a robust threshold
    full_train_tensor = torch.from_numpy(X_train_scaled.astype(np.float64)).to(device)
    
    with torch.no_grad():
        if is_parallel:
            train_log_probs = model.module.log_prob(inputs=full_train_tensor)
        else:
            train_log_probs = model.log_prob(inputs=full_train_tensor)

    # Move to CPU for numpy operations
    threshold = np.percentile(train_log_probs.cpu().numpy(), threshold_percentile)
    print(f"\nAnomaly threshold (log probability): {threshold:.4f}")

    predictions = log_probs_tensor.numpy() < threshold
    return predictions, log_probs_tensor.numpy()


X_test_tensor = torch.from_numpy(X_test_scaled.astype(np.float64))
test_dataset = TensorDataset(X_test_tensor)
test_loader = DataLoader(test_dataset, batch_size=batch_size)

anomaly_predictions, log_probabilities = detect_anomalies(model, test_loader)

correct_predictions = (anomaly_predictions == y_test)
accuracy = np.mean(correct_predictions)
true_positives = np.sum(anomaly_predictions & (y_test == 1))
false_positives = np.sum(anomaly_predictions & (y_test == 0))
recall = true_positives / np.sum(y_test == 1) if np.sum(y_test == 1) > 0 else 0

print("\n--- Performance Metrics ---")
print(f"Overall Accuracy: {accuracy * 100:.2f}%")
print(f"Fraud cases detected (Recall): {recall * 100:.2f}% ({true_positives}/{np.sum(y_test==1)})")
print(f"Normal cases flagged as fraud (False Positives): {false_positives}/{np.sum(y_test==0)}")



Loading best model from 'best_model.pth' for evaluation.
Wrapping loaded model with DataParallel for evaluation.

Anomaly threshold (log probability): -23.8393

--- Performance Metrics ---
Overall Accuracy: 84.46%
Fraud cases detected (Recall): 12.65% (115/909)
Normal cases flagged as fraud (False Positives): 323/6279
